In [ ]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [ ]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import accuracy_score

current_dir = '/kaggle/input/2023sumdpl302m'
# Đường dẫn tới thư mục chứa dữ liệu ảnh
image_folder = os.path.join(current_dir, "devset_images/devset_images")

# Đường dẫn tới file CSV chứa nhãn của các ảnh
csv_file = os.path.join(current_dir, "devset_images_gt.csv")

# Đọc dữ liệu từ file CSV
data = pd.read_csv(csv_file)

In [ ]:
data

In [ ]:
import torch
from torch.utils.data import Dataset
from PIL import Image

class FloodDataset(Dataset):
    def __init__(self, row, exact_dir, fold="train", transform=None):
        self.row = row
        self.transform = transform
        self.exact_dir = exact_dir
        self.fold = fold

    def __len__(self):
        return len(self.row.id)

    def __getitem__(self, idx):
        image_path = os.path.join(current_dir, self.exact_dir,f'{list(self.row.id)[idx]}.jpg')
        if os.path.exists(image_path):
            image = Image.open(image_path)
        else:
            image_path = os.path.join(current_dir, self.exact_dir,f'{list(self.row.id)[idx]}.png')
            if os.path.exists(image_path):
                image = Image.open(image_path)
            else:
                image_path = os.path.join(current_dir, self.exact_dir,f'{list(self.row.id)[idx]}.gif')
                image = Image.open(image_path)
        if self.transform:
            image = self.transform[self.fold](image)
            
        if (self.fold == "test"):
            return image, list(self.row.id)[idx]
        label = list(self.row.label)[idx]
        return image, label

In [ ]:
dataset = FloodDataset(data,exact_dir = "devset_images/devset_images", transform= None)

In [ ]:
# # Initialize variables to accumulate sums
# sum_channels = torch.zeros(3)
# sum_channels_squared = torch.zeros(3)
# transform = transforms.ToTensor()

# # Iterate over the dataset to compute sums
# for image, _ in dataset:
#     image = transform(image)
#     sum_channels += torch.sum(image, dim=(1, 2))  # Sum the values in each channel
#     sum_channels_squared += torch.sum(image ** 2, dim=(1, 2))  # Sum the squared values in each channel

# # Calculate the mean and standard deviation
# num_images = len(dataset)
# mean = sum_channels / (num_images * image.size(1) * image.size(2))
# std = torch.sqrt((sum_channels_squared / (num_images * image.size(1) * image.size(2))) - mean ** 2)

# print("Mean:", mean)
# print("Std:", std)

In [ ]:
print(f'label {dataset[13][1]}')
dataset[13][0]

In [ ]:
unique_labels = data["label"].unique()
print(unique_labels)

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models
import transformers
from transformers import ViTImageProcessor, ViTForImageClassification
from torchvision.models.resnet import BasicBlock, ResNet18_Weights

def get_model_resnet(num_classes):
    model_ft = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
    num_fts = model_ft.fc.in_features
    # Here the size of each output sample is set to 2.
    # Alternatively, it can be generalized to nn.Linear(num_ftrs, len(class_names)).
    # Change fc network to other custome fc network
    model_ft.fc = nn.Linear(num_fts, num_classes)  
    
    return model_ft

def get_model_resnet_with_freeze_weights(num_classes):
    model_ft = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
    num_fts = model_ft.fc.in_features
    # Here the size of each output sample is set to 2.
    # Change fc network to other custome fc network
    # model_ft.fc = nn.Linear(num_fts,num_class)
    model_ft.fc = nn.Identity()

    # freeze the weights
    for param in model_ft.parameters():
        param.requires_grad = False
        
    # Add a new linear layer
    model_ft.fc = nn.Sequential(
        nn.Linear(num_fts, 64),
        nn.ReLU(inplace = True),
        nn.Linear(64, num_classes)
    )
       
    # only train the fc_new layer
    for param in model_ft.fc.parameters():
        param.requires_grad = True
        
    return model_ft

def get_model_vit(num_classes):
    # processor for the input image
    processor = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224')

    # model
    model = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224')
    
    # get in feature space for classifications
    num_fts = model.classifier.in_features
    
    # change the classifier layer
    model.classifier = nn.Sequential(
        # nn.Linear(num_fts, 64),
        # nn.ReLU(inplace=True),
        # nn.Linear(64, num_classes)
        nn.Linear(num_fts, num_classes)
    )
    return model

def get_model_vit_with_freeze_weights(num_classes):
    # processor for the input image
    processor = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224')

    # model
    model = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224')
    
    # get in feature space for classifications
    num_fts = model.classifier.in_features
    
    for param in model.parameters():
        param.requires_grad = False
        
    # change the classifier layer
    model.classifier = nn.Sequential(
        # nn.Linear(num_fts, 64),
        # nn.ReLU(inplace=True),
        # nn.Linear(64, num_classes)
        nn.Linear(num_fts, num_classes)
    )
    
    for param in model.classifier.parameters():
        param.requires_grad = True
    
    return model

In [ ]:
import torch
from torchvision import transforms
from torch.utils.data import DataLoader

# loss function
criterion = torch.nn.CrossEntropyLoss() 
model = get_model_vit(num_classes=2)

device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')

# model to gpu
model.to(device)
lr= 1e-5

optimizer = torch.optim.Adam(model.parameters(), lr)
# auto decrease learning rate
exp_lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer=optimizer, step_size=4, gamma=0.1)

In [ ]:
transform = {"train": transforms.Compose([
    transforms.Resize((224, 224)),
#     transforms.RandomHorizontalFlip(),  # Randomly flip the image horizontally
    transforms.ToTensor(),
#     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    transforms.Normalize(mean = (0.5, 0.5, 0.5), std = (0.5, 0.5, 0.5))
]), 
   "test": transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
#     transforms.Normalize(mean = (0.4343, 0.4377, 0.4110), std = (0.2764, 0.2774, 0.2992)) 
    transforms.Normalize(mean = (0.5, 0.5, 0.5), std = (0.5, 0.5, 0.5))
   ])
}

dataset = FloodDataset(data, exact_dir = "devset_images/devset_images", transform=transform, fold = "train")

train_size = int(0.8 * len(dataset))
test_size = int(0.2 * len(dataset))
train_set, val_set = torch.utils.data.random_split(dataset, [train_size, test_size])

In [ ]:
from tqdm import tqdm
batch_size = 32
num_epochs = 4
# trainloader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=2)
trainloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=2)
train_size = len(dataset)
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    running_corrects=0
    for i, (inputs, labels) in tqdm(enumerate(trainloader)):
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs.logits, labels)
        
        _, preds = torch.max(outputs.logits, 1)
        running_corrects += torch.sum(preds == labels.data)
        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    running_loss = running_loss / (train_size // batch_size)
    running_corrects = running_corrects / train_size
    print(f"Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(trainloader)}], Loss: {running_loss/100:.4f}, Acc: {running_corrects:.4f}")
    
    # Evaluation
#     model.eval()
#     correct = 0
#     total = 0

#     testloader = torch.utils.data.DataLoader(val_set, batch_size=64, shuffle=False, num_workers=2)
#     with torch.no_grad():
#         for images, labels in testloader:
#             images = images.to(device)
#             labels = labels.to(device)
#             outputs = model(images)
#             _, predicted = torch.max(outputs.logits, 1)
#             total += labels.size(0)
#             correct += (predicted == labels).sum().item()

#     accuracy = 100 * correct / total
#     print(f"Accuracy on test set: {accuracy:.2f}%")

In [ ]:
test_data = pd.read_csv("/kaggle/input/2023sumdpl302m/test.csv")
test_data['id'] = test_data.image_id

In [ ]:
answer = {"id": [], "label": []}

In [ ]:
testset = FloodDataset(test_data, exact_dir = 'testset_images/testset_images',fold="test", transform=transform)
# Evaluation
model.eval()
correct = 0
total = 0

testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False, num_workers=2)
with torch.no_grad():
    for images, id in testloader:
        images = images.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.logits, 1)
        id = id.tolist()
        predicted = predicted.tolist()
        answer['id'].extend(id)
        answer['label'].extend(predicted)

In [ ]:
answer_df = pd.DataFrame.from_dict(answer)

In [ ]:
answer_df.drop_duplicates(subset='id').to_csv("submission.csv", index=Falsetransform